# The SO4 DMRG

The bilinear-biquadratic Hamiltonian is given by

$$
H_{b b}=\sum_i\left[J \sum_{a<b} L_i^{a b} L_{i+1}^{a b}+K\left(\sum_{a<b} L_i^{a b} L_{i+1}^{a b}\right)^2\right]
$$

For $n=4$ case, we can factorized it as $SO(4) \simeq SU(2)\times SU(2)$. Thus we can consider a spin-orbital $S=T=1/2$ two-leg spin ladder to implement the $SO(4)$ vectors and generators. We introduce

$$
\left|n^{1,2}\right\rangle=\frac{e^{ \pm i \pi / 4}}{\sqrt{2}}(|\uparrow, \uparrow\rangle \mp|\downarrow, \downarrow\rangle),
$$
and
$$
\left|n^{3,4}\right\rangle=\frac{e^{\mp i \pi / 4}}{\sqrt{2}}(|\downarrow, \uparrow\rangle \mp|\uparrow, \downarrow\rangle)
$$
where the $\ket{\sigma,\tau}$ denotes the spin and orbital directions. Moreover, the $SO(4)$ generators are defined by

$$
\begin{aligned}
& L^{12}=-T^z-S^z, \quad L^{13}=T^x-S^x, \quad L^{14}=-T^y-S^y \\
& L^{23}=T^y-S^y, \quad L^{24}=T^x+S^x, \quad L^{34}=T^z-S^z
\end{aligned}
$$

Which is a complex representation. 

We will have biquadratic terms in the Hamiltonian, following the spirit of $SO(6)$, we will finally have $4^2 = 16$ operators to represent both the bilinear and biquadratic terms. 

In [2]:
import numpy as np
import numpy.linalg as LA

def get_so4_opr_list():
    sigmax = np.array([[0, 1], [1, 0]])
    sigmay = np.array([[0, -1j], [1j, 0]])
    sigmaz = np.array([[1, 0], [0, -1]])
    id = np.eye(2)

    Sx = 0.5 * sigmax
    Sy = 0.5 * sigmay
    Sz = 0.5 * sigmaz
    Sp = Sx + 1j * Sy
    Sm = Sx - 1j * Sy

    L1 = -np.kron(Sz, id) - np.kron(id, Sz)
    L2 = -np.kron(Sx, id) + np.kron(id, Sx)
    L3 = -np.kron(Sy, id) - np.kron(id, Sy)
    L4 = -np.kron(Sy, id) + np.kron(id, Sy)
    L5 = +np.kron(Sx, id) + np.kron(id, Sx)
    L6 = -np.kron(Sz, id) + np.kron(id, Sz)

    Loprs = [L1, L2, L3, L4, L5, L6]
    coe_list = []

    for a in range(6):
        for b in range(6):
            LiLi = Loprs[a] @ Loprs[b]
            Amat = np.zeros((16, len(Loprs)), dtype=complex)
            B = LiLi.reshape(-1,1)
            for l in range(len(Loprs)):
                Amat[:,l] = Loprs[l].reshape(-1,1)[:,0]
            pcoe, resi, rank, sing = LA.lstsq(Amat, B, rcond=None)
            if len(resi)!=0 and resi[0]>1e-10:
                Loprs.append(LiLi)
                pcoe = np.append(np.zeros((len(Loprs)-1, 1)),1).reshape(len(Loprs),1)
                coe_list.append(pcoe)
            else:
                coe_list.append(pcoe)

    coe_list_new = []
    for a in range(6):
        for b in range(6):
            LiLi = Loprs[a] @ Loprs[b]
            Amat = np.zeros((16, len(Loprs)), dtype=complex)
            B = LiLi.reshape(-1,1)
            for l in range(len(Loprs)):
                Amat[:,l] = Loprs[l].reshape(-1,1)[:,0]
            pcoe = LA.solve(Amat, B)
            coe_list_new.append(pcoe)
    
    coe_list = coe_list_new

    for i in range(len(coe_list)):
        coe_list[i] = coe_list[i].reshape(16)

    def pvec(a,b):
        return coe_list[6*a+b] #a,b=0,1,2,3,4,5
    
    cmn = np.zeros((16,16), dtype=complex)

    P = dict()
    for a in range(6):
        for b in range(6):
            P[(a,b)] = pvec(a,b)
    
    for m in range(16):
        for n in range(16):
            for a in range(6):
                for b in range(6):
                    cmn[m,n] += P[(a,b)][m] * P[(a,b)][n]

    return Loprs, cmn

In [4]:
L_operators, c_mn = get_so4_opr_list()

for i in range(len(L_operators)):
    print("the", i, "th operator is")
    print(L_operators[i])
    print('')

the 0 th operator is
[[-1. -0. -0. -0.]
 [-0.  0. -0.  0.]
 [-0. -0.  0.  0.]
 [-0.  0.  0.  1.]]

the 1 th operator is
[[ 0.   0.5 -0.5  0. ]
 [ 0.5  0.   0.  -0.5]
 [-0.5  0.   0.   0.5]
 [ 0.  -0.5  0.5  0. ]]

the 2 th operator is
[[-0.-0.j  -0.+0.5j -0.+0.5j -0.-0.j ]
 [-0.-0.5j -0.-0.j  -0.-0.j  -0.+0.5j]
 [-0.-0.5j -0.-0.j  -0.-0.j  -0.+0.5j]
 [-0.-0.j  -0.-0.5j -0.-0.5j -0.-0.j ]]

the 3 th operator is
[[0.+0.j  0.-0.5j 0.+0.5j 0.+0.j ]
 [0.+0.5j 0.+0.j  0.+0.j  0.+0.5j]
 [0.-0.5j 0.+0.j  0.+0.j  0.-0.5j]
 [0.+0.j  0.-0.5j 0.+0.5j 0.+0.j ]]

the 4 th operator is
[[0.  0.5 0.5 0. ]
 [0.5 0.  0.  0.5]
 [0.5 0.  0.  0.5]
 [0.  0.5 0.5 0. ]]

the 5 th operator is
[[ 0.  0.  0.  0.]
 [ 0. -1.  0. -0.]
 [ 0.  0.  1.  0.]
 [ 0. -0.  0.  0.]]

the 6 th operator is
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]]

the 7 th operator is
[[ 0.  -0.5  0.5  0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.  -0.5  0.5  0. ]]

the 8 th operator is
[[0.+0.j  0.-0.5j 0.-0.5j 0

To distinguish the on-site operators and the original $SO(4)$ generators, we give hats to the on-site operators. They are

$$
\hat{L}^0=\left(\begin{array}{cccc}
-1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1
\end{array}\right), \quad 
\hat{L}^1=\frac{1}{2}\left(\begin{array}{cccc}
0 & 1 & -1 & 0\\
1 & 0 & 0 & -1 \\
-1 & 0 & 0 & 1 \\
0 & -1 & 1 & 0 
\end{array}\right), \quad
\hat{L}^2=\frac{1}{2}\left(\begin{array}{cccc}
0 & i & i & 0\\
-i & 0 & 0 & i \\
-i & 0 & 0 & i \\
0 & -i & -i & 0 
\end{array}\right), \quad
\hat{L}^3=\frac{1}{2}\left(\begin{array}{cccc}
0 & -i & i & 0\\
i & 0 & 0 & i \\
-i & 0 & 0 & -i \\
0 & -i & i & 0 
\end{array}\right),
$$

$$
\hat{L}^4=\frac{1}{2}\left(\begin{array}{cccc}
0 & 1 & 1 & 0\\
1 & 0 & 0 & 1 \\
1 & 0 & 0 & 1 \\
0 & 1 & 1 & 0 
\end{array}\right), \quad 
\hat{L}^5\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0
\end{array}\right), \quad
\hat{L}^6=\left(\begin{array}{cccc}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1
\end{array}\right), \quad
\hat{L}^7=\frac{1}{2}\left(\begin{array}{cccc}
0 & -1 & 1 & 0\\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & -1 & 1 & 0 
\end{array}\right),
$$

$$
\hat{L}^8=\frac{1}{2}\left(\begin{array}{cccc}
0 & -i & -i & 0\\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & -i & -i & 0 
\end{array}\right), \quad 
\hat{L}^9=\frac{1}{2}\left(\begin{array}{cccc}
0 & i & -i & 0\\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & -i & i & 0 
\end{array}\right), \quad
\hat{L}^{10}=\frac{1}{2}\left(\begin{array}{cccc}
0 & -1 & -1 & 0\\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 1 & 1 & 0 
\end{array}\right), \quad
\hat{L}^{11}=\frac{1}{2}\left(\begin{array}{cccc}
1 & 0 & 0 & -1\\
0 & 1 & -1 & 0 \\
0 & -1 & 1 & 0 \\
-1 & 0 & 0 & 1 
\end{array}\right),
$$

$$
\hat{L}^{12}=\frac{1}{2}\left(\begin{array}{cccc}
0 & 0 & 0 & 0\\
0 & i & i & 0 \\
0 & -i & -i & 0 \\
0 & 0 & 0 & 0 
\end{array}\right), \quad 
\hat{L}^{13}=\frac{1}{2}\left(\begin{array}{cccc}
i & 0 & 0 & i\\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
-i & 0 & 0 & -i 
\end{array}\right), \quad
\hat{L}^{14}=\frac{1}{2}\left(\begin{array}{cccc}
1 & 0 & 0 & -1\\
0 & 1 & 1 & 0 \\
0 & 1 & 1 & 0 \\
-1 & 0 & 0 & 1 
\end{array}\right), \quad
\hat{L}^{15}=\frac{1}{2}\left(\begin{array}{cccc}
1 & 0 & 0 & 1\\
0 & 1 & -1 & 0 \\
0 & -1 & 1 & 0 \\
1 & 0 & 0 & 1 
\end{array}\right),
$$

Then we are going to make the biquadratic terms represented by these 16 on-site operators. That is the expansion

$$
\left(\sum_{a,b=1}^6 L^a(i) \otimes L^b(i+1)\right) = \sum_{m,n=1}^{16} c_{mn} \hat{L}^m(i)\otimes \hat{L}^n(i+1)
$$

In the latest python block, we have this decomposition done
$$
L^a(i)L^b(i) = \sum_{m=1}^{16} p_m(a,b) \hat{L}^m(i)
$$

Then we have
$$
L^a(i)L^b(i) \otimes L^a(i+1)L^b(i+1) = \sum_{m=1}^{16}\sum_{n=1}^{16} p_m(a,b)p_n(a,b) \hat{L}^m(i) \otimes \hat{L}^n(i+1)
$$

By defining 
$$
c_{mn} = \sum_{a,b=1}^6 p_m(a,b) p_n(a,b), 
$$
everything follow the $SO(6)$ case. 

We notice that there are 2 diagonal matrices in the $SO(4)$ generators, they are

$$
\hat{L}^0=\left(\begin{array}{cccc}
-1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1
\end{array}\right), \quad 
\hat{L}^5\left(\begin{array}{cccc}
0 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0
\end{array}\right)
$$

The Cartan sub-algebra shows up automatically, it is 2-dimensional and we are allowed to have a $U(1)\times U(1)$ good quantum number here. 

In [ ]:
from tenpy.models.model import CouplingModel, MPOModel
from tenpy.networks.site import SpinSite, FermionSite, Site
from tenpy.models.lattice import Chain
from tenpy.networks.mps import MPS
from tenpy.networks.mpo import MPO
import tenpy.linalg.np_conserved as npc
from tenpy.linalg.charges import LegCharge, ChargeInfo
from tenpy.algorithms import dmrg

In [ ]:
class SO4Site(Site):
    def __init__(self, so4g, cons_N=None, cons_S=None):
        self.conserve = [cons_N, cons_S]
        self.cons_N = cons_N
        self.cons_S = cons_S
        self.so4g = so4g
        
        if self.cons_N is None and self.cons_S == 'U1':
            chinfo = npc.ChargeInfo([1, 1], ['S', 'T'])
            leg = npc.LegCharge.from_qflat(chinfo, [[-1, 0], [0, -1], [0, 1], [1, 0]])
        elif self.cons_N is None and self.cons_S is None:
            leg = npc.LegCharge.from_trivial(4)
        
        ops = dict()
        for i in range(len(self.so4g)):
            ops['L{}'.format(i)] = self.so4g[i]

        names = ['1u2u', '1d2u', '1u2d', '1d2d']
        Site.__init__(self, leg, names, **ops)

    def __repr__(self):
        return "trivial site for 16 so4 generators"